In [1]:
##Load metadata from file

import ast

d = {}
nary = {}
with open("metadata_trial4.txt", 'r') as f:
    for line in f:
        d = (ast.literal_eval(line))
        nary.update({d['id'] : d})

In [2]:
#Filter terms that have children and aspect

filnary = {}
kilnary = {}
nochild = {}

for key, value in nary.items():
    if value.get('children') != None:
        filnary = ({'id' : value['id'], 'name' : value['name'], 'aspect' : value['aspect'], 
                    'children' : value['children']})
        kilnary.update({key : filnary})
    else:
        filnary = ({'id' : value['id'], 'name' : value['name'], 'aspect' : value['aspect']})
        nochild.update({key : filnary})

In [3]:
#import owlready2

import types
from owlready2 import *
onto = get_ontology("http://test.org/onto.owl")

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


In [12]:
#define core classes

with onto:
    class therapeutic_agent(Thing):
        namespace = onto
    class biological_process(Thing):
        namespace = onto
    class molecular_function(Thing):
        namespace = onto
    class drug(Thing):
        namespace = onto

In [5]:
#classmaker function to make FTC classes

def classmaker(term, parentclass):
                classnameA = 'GOA' + str(term)
                classnameP = 'GOP' + str(term)
                with onto:
                    classname = types.new_class(classnameA, (parentclass,))
                    classname = types.new_class(classnameP, (parentclass,))
                return 0

In [6]:
#Using classmaker to make FTC classes

for key, value in kilnary.items():
    if value['aspect'] == 'biological_process':
        for item in value['children']:
            term = str(value['id'][3:])
            if item['relation'] == 'positively_regulates' and item['id'] in kilnary:
                #classmaker(term, therapeutic_agent)
            #if item['id'] in kilnary:
                classmaker(str(item['id'][3:]), therapeutic_agent)
                #print('biological process', value['id'], 'positively regulates', item['id'])
            if item['relation'] == 'negatively_regulates' and item['id'] in kilnary:
                #classmaker(term, therapeutic_agent)
            #if item['id'] in kilnary:
                classmaker(str(item['id'][3:]), therapeutic_agent)
                #print('biological process', value['id'], 'negatively regulates', item['id'])
            if item['relation'] == 'regulates' and item['id'] in kilnary:
                #classmaker(term, therapeutic_agent)
            #if item['id'] in kilnary:
                classmaker(str(item['id'][3:]), therapeutic_agent)
                #print('biological process', value['id'], 'regulates', item['id'])
                
    if value['aspect'] == 'molecular_function':
        for item in value['children']:
            if item['relation'] == 'positively_regulates' and item['id'] in kilnary:
                #classmaker(term, therapeutic_agent)
            #if item['id'] in kilnary:
                classmaker(str(item['id'][3:]), therapeutic_agent)
                #print('molecular function', value['id'], 'positively regulates', item['id'])
            if item['relation'] == 'negatively_regulates' and item['id'] in kilnary:
                #classmaker(term, therapeutic_agent)
            #if item['id'] in kilnary:
                classmaker(str(item['id'][3:]), therapeutic_agent)
                #print('molecular function', value['id'], 'negatively regulates', item['id'])
            if item['relation'] == 'regulates' and item['id'] in kilnary:
                #classmaker(term, therapeutic_agent)
            #if item['id'] in kilnary:
                classmaker(str(item['id'][3:]), therapeutic_agent)
                #print('molecular function', value['id'], 'regulates', item['id'])

In [6]:
#number of child classes linked by 'regulates'

print(len(kilnary.items()), len(nochild.items()), len(list(onto.classes())))
c=0
l = []

for key, value in kilnary.items():
    for item in value['children']:
        if (item['relation'] == 'positively_regulates' or item['relation'] == 'negatively regulates' or item['relation'] == 'regulates') and (value['aspect'] == 'biological_process' or value['aspect'] == 'molecular_function') and item['id'] not in l:
            l.append(item['id'])
            c+=1

print(c)        

23318 24030 0
6914


In [13]:
#make bp and mf classes

for key, value in nary.items():

    if value.get('aspect') != None and value['aspect'] == 'biological_process':
        with onto:
            classname = types.new_class(key, (biological_process,))
    if value.get('aspect') != None and value['aspect'] == 'molecular_function':
        with onto:
            classname = types.new_class(key, (molecular_function,))

In [14]:
#assign appropriate labels to classes

x = list(onto.classes())
for y in x:
    term = 'GO:' + str(y)[8:]
    if str(y)[7] == ':' and term in kilnary:
        y.label = nary[str(y)[5:]]['name']
    if str(y)[7] == 'A' and term in kilnary :
        y.label = 'anti_' + kilnary[term]['name'] + '_agent'
    
    if str(y)[7] == 'P' and term in kilnary :
        y.label = 'pro_' + kilnary[term]['name'] + '_agent'

In [15]:
#list of ids from the created classes in the two text formats

ids = []
ontoids = []
for cl in list(onto.classes()):
    cl = str(cl)
    ontoids.append(cl)
    clt = cl[5:7] +':' + cl[8:]
    ids.append(clt)

In [16]:
#propertymaker function for making properties

def propertymaker(pname, pdomain, prange):
    with onto:
            classname = types.new_class((pname), (ObjectProperty,), )
    return 0

In [17]:
#parsing biological process and molecular function classes

reldict = {}


for key, value in nary.items():            
    if value.get('children') != None and value['id'] in ids:
            
        for item in value['children']:
            if reldict.get(item['relation']) == None:
                reldict[item['relation']] = [[],[]]
                propertymaker(item['relation'], 'void', 'void')
        for item in value['children']:
            if item['id'] in ids:
                reldict[item['relation']][0].append(ids[ids.index(value['id'])])
                reldict[item['relation']][1].append(ids[ids.index(item['id'])])
        

In [19]:
#Making subproperties positively_regulates and negatively_regulates

with onto:
    class regulates(ObjectProperty):
        pass
    class positively_regulates(regulates):
        pass
    class negatively_regulates(regulates):
        pass

In [20]:
#converting string to object in reldict lists

for key, value in reldict.items():
    for item in value[0]:
        value[0][value[0].index(item)] = getattr(onto, item)
    for item in value[1]:
        value[1][value[1].index(item)] = getattr(onto, item)


In [50]:
#setting relations

for key, value in reldict.items():
    for item in value[0]:
        setattr(item, key, [value[1][value[0].index(item)]])

In [ ]:
reldict.keys()

In [51]:
onto.save(file = "testowl2", format = "rdfxml")